In [8]:
import os
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Attachment, FileContent, FileName, FileType, Disposition
import base64

def export_source_file_data_to_excel():
    try:
        # Constants
        db_file = "conversion.db"
        reports_folder = "reports/"
        
        # Get current date
        today_date = datetime.now().strftime("%Y-%m-%d")
        
        # Create reports folder if it doesn't exist
        os.makedirs(reports_folder, exist_ok=True)

        # Connect to SQLite database
        conn = sqlite3.connect(db_file)

        # Query to select data from SourceFile table for today's date
        query = f"SELECT * FROM SourceFile WHERE substr(updated_datetime, 1, 10) = '{today_date}'"
        
        # Read data into a DataFrame
        df = pd.read_sql_query(query, conn)

        # Close the database connection
        conn.close()

        # Write DataFrame to Excel file
        excel_file_path = os.path.join(reports_folder, f"Report_{today_date}.xlsx")
        df.to_excel(excel_file_path, index=False)

        print(f"Data exported to Excel file: {excel_file_path}")
        return excel_file_path
    except Exception as e:
        print(f"Error exporting data to Excel file: {str(e)}")
        return None

def send_email_notification(total_files, processed_files, failed_files, deleted_files, excel_file_path):
    try:
        # Constants
        sender_email = 'your_email@gmail.com'  # Replace with sender's email
        recipient_emails = ['recipient1@gmail.com', 'recipient2@gmail.com']  # Replace with recipient's email(s)
        cc_emails = ['cc1@gmail.com', 'cc2@gmail.com']  # Replace with CC email(s)
        sendgrid_api_key = 'your_sendgrid_api_key'  # Replace with your SendGrid API key

        # Get current date
        today_date = datetime.now().strftime("%Y-%m-%d")

        # Create email message
        message = Mail(
            from_email=sender_email,
            to_emails=recipient_emails,
            cc_emails=cc_emails,
            subject='Daily Status Report',
            html_content=f'''
                <p>Date: {today_date}</p>
                <p>Total files: {total_files}</p>
                <p>Processed files: {processed_files}</p>
                <p>Failed files: {failed_files}</p>
                <p>Deleted files: {deleted_files}</p>
            '''
        )

        # Attach Excel file
        with open(excel_file_path, 'rb') as f:
            data = f.read()
            f.close()

        encoded_file = base64.b64encode(data).decode()
        attachment = Attachment(
            FileContent(encoded_file),
            FileName(f"Report_{today_date}.xlsx"),
            FileType('application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'),
            Disposition('attachment')
        )
        message.attachment = attachment

        # Send email using SendGrid API
        sg = SendGridAPIClient(sendgrid_api_key)
        response = sg.send(message)

        print("Email notification sent successfully.")

    except Exception as e:
        print(f"Error sending email notification: {str(e)}")

def get_file_count(folder):
    try:
        count = sum(len(files) for _, _, files in os.walk(folder))
        return count
    except Exception as e:
        print(f"Error counting files in {folder}: {str(e)}")
        return 0

def main():
    try:
        # Constants
        input_folder = "input/"
        completed_folder = "completed/"
        failed_folder = "failed/"
        deleted_folder = "deleted/"

        # Get current date
        today_date = datetime.now().strftime("%Y-%m-%d")

        # Get file counts
        total_files = get_file_count(input_folder + today_date)
        processed_files = get_file_count(completed_folder + today_date)
        failed_files = get_file_count(failed_folder)
        deleted_files = get_file_count(deleted_folder + today_date)

        # Export SourceFile data to Excel
        excel_file_path = export_source_file_data_to_excel()

        # Send email notification
        send_email_notification(total_files, processed_files, failed_files, deleted_files, excel_file_path)

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'sendgrid'

In [ ]:
import os
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Attachment, FileContent, FileName, FileType, Disposition
import base64

def get_file_count(db_file, table, status_column, status, date_column, date):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Query to get file count based on status and date
        query = f"SELECT COUNT(*) FROM {table} WHERE {status_column} = ? AND date({date_column}) = ?"
        cursor.execute(query, (status, date))
        count = cursor.fetchone()[0]

        # Close the database connection
        conn.close()

        return count
    except Exception as e:
        print(f"Error getting file count from {table}: {str(e)}")
        return 0

def export_source_file_data_to_excel(db_file):
    try:
        # Constants
        reports_folder = "reports/"
        
        # Get current date
        today_date = datetime.now().strftime("%Y-%m-%d")
        
        # Create reports folder if it doesn't exist
        os.makedirs(reports_folder, exist_ok=True)

        # Connect to SQLite database
        conn = sqlite3.connect(db_file)

        # Query to select data from SourceFile table for today's date
        query = f"SELECT * FROM SourceFile WHERE date(updated_datetime) = '{today_date}'"
        
        # Read data into a DataFrame
        df = pd.read_sql_query(query, conn)

        # Close the database connection
        conn.close()

        # Write DataFrame to Excel file
        excel_file_path = os.path.join(reports_folder, f"Report_{today_date}.xlsx")
        df.to_excel(excel_file_path, index=False)

        print(f"Data exported to Excel file: {excel_file_path}")
        return excel_file_path
    except Exception as e:
        print(f"Error exporting data to Excel file: {str(e)}")
        return None

def send_email_notification(total_files, processed_files, failed_files, deleted_files, excel_file_path):
    try:
        # Constants
        sender_email = 'your_email@gmail.com'  # Replace with sender's email
        recipient_emails = ['recipient1@gmail.com', 'recipient2@gmail.com']  # Replace with recipient's email(s)
        cc_emails = ['cc1@gmail.com', 'cc2@gmail.com']  # Replace with CC email(s)
        sendgrid_api_key = 'your_sendgrid_api_key'  # Replace with your SendGrid API key

        # Get current date
        today_date = datetime.now().strftime("%Y-%m-%d")

        # Create email message
        message = Mail(
            from_email=sender_email,
            to_emails=recipient_emails,
            cc_emails=cc_emails,
            subject='Daily Status Report',
            html_content=f'''
                <p>Date: {today_date}</p>
                <p>Total files: {total_files}</p>
                <p>Processed files: {processed_files}</p>
                <p>Failed files: {failed_files}</p>
                <p>Deleted files: {deleted_files}</p>
            '''
        )

        # Attach Excel file
        with open(excel_file_path, 'rb') as f:
            data = f.read()
            f.close()

        encoded_file = base64.b64encode(data).decode()
        attachment = Attachment(
            FileContent(encoded_file),
            FileName(f"Report_{today_date}.xlsx"),
            FileType('application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'),
            Disposition('attachment')
        )
        message.attachment = attachment

        # Send email using SendGrid API
        sg = SendGridAPIClient(sendgrid_api_key)
        response = sg.send(message)

        print("Email notification sent successfully.")

    except Exception as e:
        print(f"Error sending email notification: {str(e)}")

def main():
    try:
        # Constants
        db_file = "conversion.db"

        # Get current date
        today_date = datetime.now().strftime("%Y-%m-%d")

        # Get file counts
        total_files = get_file_count(db_file, "SourceFile", "status", "Ready", "updated_datetime", today_date)
        processed_files = get_file_count(db_file, "ProcessedFile", "status", "Completed", "chunk_created_datetime", today_date)
        failed_files = get_file_count(db_file, "SourceFile", "status", "Failed", "updated_datetime", today_date)
        deleted_files = get_file_count(db_file, "SourceFile", "status", "Deleted", "updated_datetime", today_date)

        # Export SourceFile data to Excel
        excel_file_path = export_source_file_data_to_excel(db_file)

        # Send email notification
        send_email_notification(total_files, processed_files, failed_files, deleted_files, excel_file_path)

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()


In [ ]:
import os
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def get_file_count(db_file, table, status_column, status, date_column, date):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Query to get file count based on status and date
        query = f"SELECT COUNT(*) FROM {table} WHERE {status_column} = ? AND date({date_column}) = ?"
        cursor.execute(query, (status, date))
        count = cursor.fetchone()[0]

        # Close the database connection
        conn.close()

        return count
    except Exception as e:
        print(f"Error getting file count: {str(e)}")
        return None

def send_email(sender_email, receiver_email, password, subject, body, attachment_path=None):
    try:
        # Setup the MIME
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = receiver_email
        message['Subject'] = subject

        # Add body to email
        message.attach(MIMEText(body, 'plain'))

        if attachment_path:
            # Open PDF file in binary mode
            with open(attachment_path, 'rb') as attachment:
                # Add file as application/octet-stream
                # Email client can usually download this automatically as attachment
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment.read())

            # Encode file in ASCII characters to send by email    
            encoders.encode_base64(part)
            
            # Add header as key/value pair to attachment part
            part.add_header('Content-Disposition', f'attachment; filename= {os.path.basename(attachment_path)}')
            
            # Add attachment to message and convert message to string
            message.attach(part)

        # Create secure connection with server and send email
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, message.as_string())

        print("Email sent successfully")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

def send_daily_status_email():
    try:
        # Database file
        db_file = "conversion.db"

        # Get today's date
        today_date = datetime.now().strftime("%Y-%m-%d")

        # Get file counts
        total_files = get_file_count(db_file, "SourceFile", "status", "Ready", "updated_datetime", today_date)
        processed_files = get_file_count(db_file, "ProcessedFile", "status", "Completed", "chunk_created_datetime", today_date)
        failed_files = get_file_count(db_file, "SourceFile", "status", "Failed", "updated_datetime", today_date)
        deleted_files = get_file_count(db_file, "SourceFile", "status", "Deleted", "updated_datetime", today_date)

        # Email content
        sender_email = "pbharat@trellissoft.ai"
        receiver_email = "bpathrabe2@gmail.com"
        password = "Kunkerochan@55555"
        subject = "Daily Status Report"
        body = f"Date: {today_date}\nTotal files: {total_files}\nProcessed files: {processed_files}\nFailed files: {failed_files}\nDeleted files: {deleted_files}"
        attachment_path = f"reports/Report_{today_date}.xlsx"

        # Send email
        send_email(sender_email, receiver_email, password, subject, body, attachment_path)
    except Exception as e:
        print(f"Error sending daily status email: {str(e)}")

if __name__ == "__main__":
    send_daily_status_email()


Error getting file count: no such table: SourceFile
Error getting file count: no such table: ProcessedFile
Error getting file count: no such table: SourceFile
Error getting file count: no such table: SourceFile
Error sending email: [Errno 2] No such file or directory: 'reports/Report_2024-04-19.xlsx'


In [ ]:
import os
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import ssl

def get_file_count(db_file, table, status_column, status, date_column, date):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Query to get file count based on status and date
        query = f"SELECT COUNT(*) FROM {table} WHERE {status_column} = ? AND substr({date_column}, 1, 10) = ?"
        cursor.execute(query, (status, date))
        count = cursor.fetchone()[0]

        # Close the database connection
        conn.close()

        return count
    except Exception as e:
        print(f"Error getting file count: {str(e)}")
        return None

def send_email(sender_email, receiver_email, password, subject, body, attachment_path=None):
    try:
        # Setup the MIME
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = receiver_email
        message['Subject'] = subject

        # Add body to email
        message.attach(MIMEText(body, 'plain'))

        if attachment_path:
            # Open PDF file in binary mode
            with open(attachment_path, 'rb') as attachment:
                # Add file as application/octet-stream
                # Email client can usually download this automatically as attachment
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment.read())

            # Encode file in ASCII characters to send by email    
            encoders.encode_base64(part)
            
            # Add header as key/value pair to attachment part
            part.add_header('Content-Disposition', f'attachment; filename= {os.path.basename(attachment_path)}')
            
            # Add attachment to message and convert message to string
            message.attach(part)

        # Create secure connection with server and send email
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, message.as_string())

        print("Email sent successfully")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

def send_daily_status_email():
    try:
        # Database file
        db_file = "conversion.db"

        # Get today's date
        today_date = datetime.now().strftime("%Y-%m-%d")

        # Get file counts
        total_files = get_file_count(db_file, "SourceFile", "status", "Ready", "updated_datetime", today_date)
        processed_files = get_file_count(db_file, "ProcessedFile", "status", "Completed", "chunk_created_datetime", today_date)
        failed_files = get_file_count(db_file, "SourceFile", "status", "Failed", "updated_datetime", today_date)
        deleted_files = get_file_count(db_file, "SourceFile", "status", "Deleted", "updated_datetime", today_date)

        # Email content
        sender_email = "bpathrabe2@gmail.com"
        receiver_email = "chatkozak@gmail.com"
        password = "mnyy msya iokj vwtm"
        subject = "Daily Status Report"
        body = f"Date: {today_date}\nTotal files: {total_files}\nProcessed files: {processed_files}\nFailed files: {failed_files}\nDeleted files: {deleted_files}"
        attachment_path = f"reports/Report_{today_date}.xlsx"

        # Send email
        send_email(sender_email, receiver_email, password, subject, body, attachment_path)
    except Exception as e:
        print(f"Error sending daily status email: {str(e)}")

if __name__ == "__main__":
    send_daily_status_email()


Email sent successfully


In [13]:
import os
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import ssl

def get_file_count(db_file, table, status_column, status, date_column, date):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Query to get file count based on status and date
        query = f"SELECT COUNT(*) FROM {table} WHERE {status_column} = ? AND substr({date_column}, 1, 10) = ?"
        cursor.execute(query, (status, date))
        count = cursor.fetchone()[0]

        # Close the database connection
        conn.close()

        return count
    except Exception as e:
        print(f"Error getting file count: {str(e)}")
        return None

def send_email(sender_email, receiver_email, cc_emails, password, subject, body, attachment_path=None):
    try:
        # Setup the MIME
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = receiver_email
        message['Cc'] = ", ".join(cc_emails) if cc_emails else ""
        message['Subject'] = subject

        # Add body to email
        message.attach(MIMEText(body, 'plain'))

        if attachment_path:
            # Open PDF file in binary mode
            with open(attachment_path, 'rb') as attachment:
                # Add file as application/octet-stream
                # Email client can usually download this automatically as attachment
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment.read())

            # Encode file in ASCII characters to send by email    
            encoders.encode_base64(part)
            
            # Add header as key/value pair to attachment part
            part.add_header('Content-Disposition', f'attachment; filename= {os.path.basename(attachment_path)}')
            
            # Add attachment to message and convert message to string
            message.attach(part)

        # Create secure connection with server and send email
        context = ssl.create_default_context()
        with smtplib.SMTP("smtp.gmail.com", 465) as server:
            server.starttls(context=context)
            server.login(sender_email, password)
            server.sendmail(sender_email, [receiver_email] + cc_emails, message.as_string())

        print("Email sent successfully")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

def send_daily_status_email():
    try:
        # Database file
        db_file = "conversion.db"

        # Get today's date
        today_date = datetime.now().strftime("%Y-%m-%d")

        # Get file counts
        total_files = get_file_count(db_file, "SourceFile", "status", "Ready", "updated_datetime", today_date)
        processed_files = get_file_count(db_file, "ProcessedFile", "status", "Completed", "chunk_created_datetime", today_date)
        failed_files = get_file_count(db_file, "SourceFile", "status", "Failed", "updated_datetime", today_date)
        deleted_files = get_file_count(db_file, "SourceFile", "status", "Deleted", "updated_datetime", today_date)

        # Email content
        sender_email = "bpathrabe2@gmail.com"
        receiver_email = "pbharat@trellissoft.ai"
        cc_emails = ["bsakshi@trellissoft.ai", "pjignesh@trellissoft.ai"] 
        password = "mnyy msya iokj vwtm"
        subject = "Daily Status Report"
        body = f"Date: {today_date}\nTotal files: {total_files}\nProcessed files: {processed_files}\nFailed files: {failed_files}\nDeleted files: {deleted_files}"
        attachment_path = f"reports/Report_{today_date}.xlsx"

        # Send email
        send_email(sender_email, receiver_email, cc_emails, password, subject, body, attachment_path)
    except Exception as e:
        print(f"Error sending daily status email: {str(e)}")

if __name__ == "__main__":
    send_daily_status_email()


Error sending email: Connection unexpectedly closed


In [17]:
import os
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import ssl

def get_file_count(db_file, table, status_column, status, date_column, date):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Query to get file count based on status and date
        query = f"SELECT COUNT(*) FROM {table} WHERE {status_column} = ? AND substr({date_column}, 1, 10) = ?"
        cursor.execute(query, (status, date))
        count = cursor.fetchone()[0]

        # Close the database connection
        conn.close()

        return count
    except Exception as e:
        print(f"Error getting file count: {str(e)}")
        return None

def send_email(sender_email, receiver_email, cc_emails, password, subject, body, attachment_path=None):
    try:
        # Setup the MIME
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = receiver_email
        message['Cc'] = ", ".join(cc_emails) if cc_emails else ""
        message['Subject'] = subject

        # Add body to email
        message.attach(MIMEText(body, 'plain'))

        if attachment_path:
            # Open PDF file in binary mode
            with open(attachment_path, 'rb') as attachment:
                # Add file as application/octet-stream
                # Email client can usually download this automatically as attachment
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment.read())

            # Encode file in ASCII characters to send by email    
            encoders.encode_base64(part)
            
            # Add header as key/value pair to attachment part
            part.add_header('Content-Disposition', f'attachment; filename= {os.path.basename(attachment_path)}')
            
            # Add attachment to message and convert message to string
            message.attach(part)

        # Create secure connection with server and send email
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, [receiver_email] + cc_emails, message.as_string())

        print("Email sent successfully")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

def send_daily_status_email():
    try:
        # Database file
        db_file = "conversion.db"

        # Get today's date
        today_date = datetime.now().strftime("%Y-%m-%d")

        # Get file counts
        total_files = get_file_count(db_file, "SourceFile", "status", "Ready", "updated_datetime", today_date)
        processed_files = get_file_count(db_file, "ProcessedFile", "status", "Completed", "chunk_created_datetime", today_date)
        failed_files = get_file_count(db_file, "SourceFile", "status", "Failed", "updated_datetime", today_date)
        deleted_files = get_file_count(db_file, "SourceFile", "status", "Deleted", "updated_datetime", today_date)

        # Email content
        sender_email = "bpathrabe2@gmail.com"
        receiver_email = "chatkozak@gmail.com"
        cc_emails = ["kunkerochan@gmail.com", "pjignesh@trellissoft.ai"] 
        password = "mnyy msya iokj vwtm"
        subject = "Daily Status Report"
        body = f"Date: {today_date}\nTotal files: {total_files}\nProcessed files: {processed_files}\nFailed files: {failed_files}\nDeleted files: {deleted_files}"
        attachment_path = f"reports/Report_{today_date}.xlsx"

        # Send email
        send_email(sender_email, receiver_email, cc_emails, password, subject, body, attachment_path)
    except Exception as e:
        print(f"Error sending daily status email: {str(e)}")

if __name__ == "__main__":
    send_daily_status_email()


Email sent successfully
